In [1]:
import numpy as np
import pandas as pd
from dijkstar import Graph, find_path
import csv
import math

#function for finding linkid, given start nodeid and end nodeid
df = pd.read_table('links.csv',delimiter=',')
df_node = pd.read_table('nodes.csv',delimiter=',')
link_df = df[['link_id','begin_node_id','end_node_id']]
def find_link_id(node1,node2):
    out = link_df[(link_df['begin_node_id']==node1)&(link_df['end_node_id']==node2)]
    if len(out) == 0:
        link_id = -1
    else:
        link_id = out['link_id'].values[0]
    return link_id

#get the linkids of a rectangle block
def area_linkid(x1,x2,y1,y2):
    linkid = []
    df1 = df.loc[(df['startX'] >x1) & (df['endX'] < x2), :]
    df2 = df.loc[(df['startY'] >y1) & (df['endY'] < y2), :]
    '''df1_temp = df1[['link_id','startX','startY','endX','endY']]
    df2_temp  =df2[['link_id','startX','startY','endX','endY']]'''
    
    s1 = pd.merge(df1, df2, on='link_id', how='inner')
    return s1
temp = area_linkid( -74.004604,-73.975537,40.737593,40.754935)

temp = temp[['link_id','startX_x','startY_x','endX_x','endY_x']]
start_points_temp = temp[['startY_x','startX_x']].as_matrix()
end_points_temp = temp[['endY_x','endX_x']].as_matrix()
linkid_temp = temp['link_id'].astype(str).as_matrix()



#big is the array of linkid
big = linkid_temp

#read file to obtain the weight for the edges
#Data is the weight matrix. Each column corresponds to the travel time of each node within that hour
#Each row corresponds to the linkid
data=pd.read_csv('targets_10X10.csv', sep=',',header=None)
data = data.values

#get the nodeid
df = pd.read_csv('links.csv')
result = pd.DataFrame([])
for i in range(0, len(big)):
    result = result.append(df.iloc[[int(big[i])-1]])
#result is the matrix of the linkid and its corresponding start node and end node
result = result.values
result = result[:,0:3]
#start node and end node
start = result[:,1]
end = result[:,2]
#get all nodes
nodes = np.concatenate((start, end), axis=0)

#helper function for checking nan
def isNaN(num):
    return num != num

#build the graph
def shortest_path_for_one_hour(time):
    graph = Graph()
    for i in range(len(start)):
        if not isNaN(data[i][time]):
            graph.add_edge(start[i],end[i], {'cost': data[i][time]})
            cost_func = lambda u, v, e, prev_e: e['cost']
    #initialize dictionary of shortest path
    ssp = {}

    for i in nodes:
        for j in nodes:
            if i != j:
                #t is a tuple of startpoint and endpoint. t:(startpoint,endpoint). note that startpoint != endpoint
                t = (i,j)
                try:
                    #run dijkstar and save the path in a dictionary
                    #Format: Key: t(tuple). Value: {nodeids in shortest path, every cost of the edge, total cost}
                    ssp[t] = find_path(graph, i, j, cost_func=cost_func)
                except:
                    #if there is no path, write no paht between startpoint and endpoint
                    ssp[t] = "No path between them"
                
    return ssp

sp = shortest_path_for_one_hour(3)

# divide region for the arc flag
nodes = pd.read_table('nodes.csv',delimiter=',')
def zone_node(x1,x2,y1,y2):
    temp = nodes.loc[(nodes['xcoord'] >=x1) & (nodes['xcoord'] < x2) & (nodes['ycoord'] >=y1) & (nodes['ycoord'] < y2)]['node_id']
    return temp

zone1 = zone_node(-73.985226,-73.975537,40.737593,40.746264)
zone2 = zone_node(-73.985226,-73.975537,40.746264,40.754935)
zone3 = zone_node(-73.994915,-73.985226,40.737593,40.746264)
zone4 = zone_node(-73.994915,-73.985226,40.746264,40.754935)
zone5 = zone_node(-74.004604,-73.994915,40.737593,40.746264)
zone6 = zone_node(-74.004604,-73.994915,40.746264,40.754935)

a = zone1.as_matrix()
b = zone2.as_matrix()
c = zone3.as_matrix()
d = zone4.as_matrix()
e = zone5.as_matrix()
f = zone6.as_matrix()
region = {1:a,2:b,3:c,4:d,5:e,6:f}

#build the arc_flags dictionary
arc_flags ={}
#Since there are 6 regions, the length of the vector should be 6
for i in result[:,0]:
    arc_flags[i] = np.zeros(6)
#Now iterate in the region dictionary
for k1 in region.keys():
    for k2 in region.keys():
        # k1 = k2 means these two nodes are in the same region
        # Because they are in the same region, that region should be set to 1
        if k1==k2:
            nodes_in_region = region[k1]
            for x in nodes_in_region:
                for y in nodes_in_region:
                    if x==y:
                        #same nodes. Done
                        pass
                    else:
                        #no edge between nodes x and y
                        if find_link_id(x,y) == -1:
                            pass
                        else:
                            #find the shortest path between node x and node y
                            nodes_in_path= sp[(x,y)][0]
                            for z in range(len(nodes_in_path)-1):
                                #set every edge between node[z] and node[z+1] to 1
                                arc_flags[find_link_id(nodes_in_path[z], nodes_in_path[z+1])][k2-1] = 1
                            #arc_flags[int(find_link_id(x,y))][k1 - 1] = 1
        else:
            #different regions
            nodes_in_region1 = region[k1]
            nodes_in_region2 = region[k2]
            
            for x in nodes_in_region1:
                for y in nodes_in_region2:
                    try:
                        #get the shortest path from the shortest path dictionary
                        nodes_in_path,c,b,e = sp[(x,y)]
                        if nodes_in_path == "No path between them":
                            pass
                        else:
                            for z in range(len(nodes_in_path)-1):
                                #set every edge between node[z] and node[z+1] to 1
                                arc_flags[int(find_link_id(nodes_in_path[z], nodes_in_path[z+1]))][k2-1] = 1
                    except:
                        pass
                        
print (arc_flags)

{5535: array([ 1.,  0.,  1.,  1.,  1.,  1.]), 7146: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 14813: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 17204: array([ 0.,  0.,  0.,  0.,  0.,  1.]), 28580: array([ 1.,  1.,  1.,  1.,  1.,  0.]), 28721: array([ 0.,  1.,  1.,  1.,  1.,  1.]), 32377: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 35889: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 40861: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 42551: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 47763: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 60034: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 61897: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 62457: array([ 1.,  1.,  0.,  0.,  0.,  0.]), 62532: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 62539: array([ 1.,  0.,  1.,  0.,  1.,  1.]), 63799: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 64201: array([ 0.,  0.,  0.,  0.,  0.,  0.]), 64290: array([ 1.,  1.,  1.,  1.,  1.,  1.]), 64826: array([ 1.,  1.,  1.,  0.,  0.,  0.]), 64838: array([ 0.,  1.,  0.,  0.,  0.,  0.]), 64839: array([ 1.,  1.,  1.,  1.,  